In [166]:
import pandas as pd
import re
import numpy as np

AADT = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vRXk2ssI2wP32cbxd7gJecku7nS9Mjim7Ed3dorQsgalcBYC6KbxpWKlx0ClBsmsgAcbf5QzQqt2tsy/pub?output=csv")

In [167]:
Car_Crash = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vRJhryMDLGWP2PxsaXiDYb5PdBN_vmZxV0aieOFUJNuD5OBBJTR927qUVRnPFBg_5iDbFgxWzDWPvC9/pub?output=csv")

In [32]:
Car_Crash.head()[["County","City"]]

,County,City
0,King,Bellevue
1,Kitsap,Bremerton
2,Spokane,Airway Heights
3,Spokane,Airway Heights
4,Grant,Mattawa


In [168]:
col1 = []
for x in Car_Crash["Primary Trafficway"]:
    #Gets the state roads numbers from the primary trafficway. Has to match "[String of nondigit text]integer" 
    #Does remove some values that don't follow road convenient of state roads. Does account for roads like 123th Highway
    if (not re.match("\D+\d+", x) == None): 
        col1.append(re.findall(r'\d+', x)[0])
    else:
        col1.append(None)

col2 = []
for x in Car_Crash["Secondary Trafficway"]:
    #Gets the state roads numbers from the secondary trafficway. Has to match "[String of nondigit text]integer" 
    #Does remove some values that don't follow road convenient of state roads. Does account for roads like 123th Highway
    if type(x) == str:
        if (not re.match("\D+\d+", x) == None):
            col2.append(re.findall(r'\d+', x)[0])
        else:
            col2.append(None)
    else:
        col2.append(None)

State_Road_Num = []
for x in np.arange(len(col1)):
    #From the 2 lists of state road numbers, it will first the state road number of the primary trafficway
    #If that is not available, it takes the state road number of the secondary trafficway
    if not col1[x] == None:
        State_Road_Num.append(col1[x])
    else:
        State_Road_Num.append(col2[x])

In [216]:
Car_Crash = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vRJhryMDLGWP2PxsaXiDYb5PdBN_vmZxV0aieOFUJNuD5OBBJTR927qUVRnPFBg_5iDbFgxWzDWPvC9/pub?output=csv")
Car_Crash.insert(7,"Primary Road Number", col1)
Car_Crash.insert(9,"Secondary Road Number", col2)
Car_Crash.insert(10, "Associated State Road Number", State_Road_Num)
Car_Crash.head()[["Primary Road Number","Secondary Road Number","Associated State Road Number"]]

,Primary Road Number,Secondary Road Number,Associated State Road Number
0,90,405,90
1,3,None,3
2,2,None,2
3,2,None,2
4,243,None,243


In [170]:
mileposts = []
for x in AADT["Location"]:
    match = re.search("(MILEPOST) (\d+.\d+)",x)
    mileposts.append(float(match.group(2)))

AADT.insert(6, "Mile Posts", mileposts)

In [212]:
Crash_AADT = []
for x in np.arange(len(Car_Crash)):
    if not Car_Crash['Associated State Road Number'][x] == None:
        if Car_Crash['Mile Post'][x] > -1:
            SR = AADT[['Mile Posts', 'AADT']].loc[AADT['StateRouteNumber'] == int(Car_Crash['Associated State Road Number'][x])]
            if len(SR) > 0:
                SR.index = np.arange(len(SR))
                Dist = 100090
                Best = -1
                for y in np.arange(len(SR)):
                    if (np.abs(SR['Mile Posts'][y]-Car_Crash['Mile Post'][x])) < Dist:
                        Best = y
                        Dist = np.abs(SR['Mile Posts'][y]-Car_Crash['Mile Post'][x])
                Crash_AADT.append(SR['AADT'][Best])
            else:
                Crash_AADT.append(None)
        else:
            R = AADT['AADT'].loc[AADT['StateRouteNumber'] == int(Car_Crash['Associated State Road Number'][0])]
            Crash_AADT.append(R.median())
    else:
        Crash_AADT.append(None)

In [217]:
Car_Crash.insert(Car_Crash.shape[1], "AADT", Crash_AADT)

In [218]:
Car_Crash.head()

,Collision Report Number,Collision Date,County,City,Jurisdiction,Agency,Primary Trafficway,Primary Road Number,Secondary Trafficway,Secondary Road Number,...,Object Struck,Damage Threshold Met,Hit and Run,Motor Vehicles Involved,Passengers Involved,Commercial Carrier Involved,School Bus Involved,Pedestrians Involved,Pedalcyclists Involved,AADT
0,EC12134,1/1/2022 12:13:00 AM,King,Bellevue,State Road,State Patrol,WB I 90,90,FROM SB I-405,405,...,barrier,Y,N,1,N,N,N,0,0,84000.0
1,EC09140,1/1/2022 12:26:00 AM,Kitsap,Bremerton,State Road,State Patrol,SR 3,3,NaN,None,...,Concrete/Jersey Barrier,Y,N,1,N,N,N,0,0,80000.0
2,EC28579,1/1/2022 12:33:00 AM,Spokane,Airway Heights,State Road,Municipal/City Police,HWY 2,2,GARFIELD RD,None,...,NaN,N,N,2,Y,N,N,0,0,5400.0
3,EC28579,1/1/2022 12:33:00 AM,Spokane,Airway Heights,State Road,Municipal/City Police,HWY 2,2,GARFIELD RD,None,...,NaN,Y,N,2,Y,N,N,0,0,5400.0
4,EC09574,1/1/2022 12:48:00 AM,Grant,Mattawa,State Road,State Patrol,SR 243,243,MORRISON ROAD,None,...,none,Y,Y,2,Y,N,N,0,0,5200.0


In [198]:
Car_Crash.iloc[3594]

Collision Report Number                      EC21598
Collision Date                  2/4/2022 12:05:00 PM
County                                         Clark
City                                       Vancouver
Jurisdiction                              State Road
Agency                                  State Patrol
Primary Trafficway                               503
Primary Road Number                             None
Secondary Trafficway            NE 65T ST           
Secondary Road Number                             65
Associated State Road Number                      65
Block Number                                     NaN
Mile Post                                        0.2
School Zone                                        N
Intersection Related                               Y
Weather Condition                           Overcast
Lighting Condition                          Daylight
Injury Severity                  No Injury Collision
Collision Type                                